In [1]:
import numpy as np
import os,math,random
from matplotlib import pyplot as plt
from scipy.stats import binned_statistic, binned_statistic_2d
from astropy.table import Table, vstack
from astropy.cosmology import FlatLambdaCDM, z_at_value
from astropy import units as u
from astropy.constants import k_B, m_p, G, M_sun
from astropy.io import fits
from astropy.coordinates import SkyCoord
from astropy.wcs import WCS
from gdpyc import GasMap, DustMap
import time
import pandas as pd

/Users/zhengyunliang/opt/anaconda3/lib/python3.9/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.26.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
cosmo = FlatLambdaCDM(Om0=0.315,H0=100)

In [3]:
Cens = Table.read('../data/DESI/CensACT.fits')
Cens['R180'] = (((10**Cens['logMh']*M_sun)/(4*np.pi/3*180*cosmo.Om(0)*3*(cosmo.H0**2)/(8*np.pi*G)))**(1./3)).to('kpc')/(1+Cens['z'])
Cens['R180_ang'] = Cens['R180']/cosmo.angular_diameter_distance(Cens['z']).to('kpc')

bcg_pos = SkyCoord(Cens['ra']*u.deg, Cens['dec']*u.deg)
xry_pos = SkyCoord(Cens['RA_X']*u.deg, Cens['DEC_X']*u.deg)
luw_pos = SkyCoord(Cens['ragroup']*u.deg, Cens['decgroup']*u.deg)

In [4]:
def create_circular_mask(h, w, center=None, radius=None):
    if center is None: # use the middle of the image
        center = (int(w/2), int(h/2))
    if radius is None: # use the smallest distance between the center and image walls
        radius = min(center[0], center[1], w-center[0], h-center[1])
    Y, X = np.ogrid[:h, :w]
    dist_from_center = np.sqrt((X - center[0])**2 + (Y - center[1])**2)
    mask = dist_from_center <= radius
    return mask

## $2 \times 2$ bins in $M_h$ and Redshift Spaces

In [5]:
step = 0.5
mbins = np.arange(14,15.1,step)
layers = 2

r180_min = np.zeros((2,len(mbins)-1))
r180_med = np.zeros((2,len(mbins)-1))
r180_max = np.zeros((2,len(mbins)-1))
r180_min_ang = np.zeros((2,len(mbins)-1))
r180_med_ang = np.zeros((2,len(mbins)-1))
r180_max_ang = np.zeros((2,len(mbins)-1))

for zz in range(2):
    for mm in range(len(mbins)-1):
        r180_min[zz,mm] = (((10**mbins[mm]*M_sun)/(4*np.pi/3*180*cosmo.Om(0)*3*cosmo.H0**2/(8*np.pi*G)))**(1./3)).to('kpc').value/(1+0.5*(zz+1))
        r180_med[zz,mm] = (((10**((mbins[mm]+mbins[mm+1])/2)*M_sun)/(4*np.pi/3*180*cosmo.Om(0)*3*cosmo.H0**2/(8*np.pi*G)))**(1./3)).to('kpc').value/(1+0.5*(zz+0.5))
        r180_max[zz,mm] = (((10**(mbins[mm+1])*M_sun)/(4*np.pi/3*180*cosmo.Om(0)*3*cosmo.H0**2/(8*np.pi*G)))**(1./3)).to('kpc').value/(1+0.5*zz)
        r180_min_ang[zz,mm] = r180_min[zz,mm]/cosmo.angular_diameter_distance(0.25+0.5*zz).to('kpc').value
        r180_med_ang[zz,mm] = r180_med[zz,mm]/cosmo.angular_diameter_distance(0.25+0.5*zz).to('kpc').value
        r180_max_ang[zz,mm] = r180_max[zz,mm]/cosmo.angular_diameter_distance(0.25+0.5*zz).to('kpc').value

## eROSITA Tiles Info

In [7]:
tiles = fits.open('../data/eROSITA/SKYMAPS_052022_MPE.fits')
tlt_select = Table(tiles[1].data)
# l_ = elt.transform_to('galactic').l.to('deg').value
elt_sel = SkyCoord(tlt_select['RA_CEN']*u.deg,tlt_select['DE_CEN']*u.deg)

Emin, Emax = 0.5, 2
zoom, zoom2 = 0.5, 4

idx_srv, d2d_srv, d3d_srv = xry_pos.match_to_catalog_sky(elt_sel)
srv_n = [i for i in set(tlt_select['SRVMAP'][idx_srv])]

In [9]:
len(srv_n), len(Cens)

(798, 73348)

In [10]:
def tlnm_name(tlnm_):
    if len(tlnm_) == 5:
        tlnm_ = '0'+tlnm_
    elif len(tlnm_) == 4:
        tlnm_ = '00'+tlnm_
    elif len(tlnm_) == 3:
        tlnm_ = '000'+tlnm_
    return tlnm_

def rescale_(mskk, ima_size):
    f_mc = interpolate.interp2d(np.arange(mskk.shape[1]), np.arange(mskk.shape[0]), mskk, kind='linear')
    mskk_rebin = f_mc(np.linspace(0, mskk.shape[1], ima_size), np.linspace(0, mskk.shape[1], ima_size))
    mskk_rebin[mskk_rebin>=0.5] = 1
    mskk_rebin[mskk_rebin<0.5] = 0
    return mskk_rebin

def mask_conta(tabs,detmsk,xx,yy,R_ang,xxs,yys,rrs):
    msk_id = np.ones(len(tabs))
    if len(rrs) > 0:
        mask_sources = []
        for s in range(len(rrs)):
            mask_sj = create_circular_mask(detmsk.shape[0], detmsk.shape[1], 
                                           center=((xxs[s]-xx+R_ang*2), (yys[s]-yy+R_ang*2)),
                                           radius=(rrs[s]/4+30/4))
            mask_sources.append(mask_sj)
            msk_id[np.sqrt((tabs['xx']-xxs[s])**2+(tabs['yy']-yys[s])**2)<(rrs[s]/4+30/4)] = 0
        mask_contamination = 1 - np.sum(np.array(mask_sources), axis=0)
        mask_contamination[mask_contamination < 1] = 0
    else:
        mask_contamination = np.ones((detmsk.shape[0], detmsk.shape[1]))
    return msk_id, mask_contamination

# Image Stacking

In [12]:
repeat_ = 51

pure_count = [[[[[]for i in range(2)] for i in range(2)] for i in range(layers)] for i in range(repeat_)]
weighted_count = [[[[[]for i in range(2)] for i in range(2)] for i in range(layers)] for i in range(repeat_)]
mask_ima = [[[[[]for i in range(2)] for i in range(2)] for i in range(layers)] for i in range(repeat_)]

pure_count_sat = [[[[[]for i in range(2)] for i in range(2)] for i in range(layers)] for i in range(repeat_)]
weighted_count_sat = [[[[[]for i in range(2)] for i in range(2)] for i in range(layers)] for i in range(repeat_)]
mask_ima_sat = [[[[[]for i in range(2)] for i in range(2)] for i in range(layers)] for i in range(repeat_)]

pure_count_sat_bq = [[[[[]for i in range(2)] for i in range(2)] for i in range(layers)] for i in range(repeat_)]
weighted_count_sat_bq = [[[[[]for i in range(2)] for i in range(2)] for i in range(layers)] for i in range(repeat_)]
mask_ima_sat_bq = [[[[[]for i in range(2)] for i in range(2)] for i in range(layers)] for i in range(repeat_)]

In [13]:
for hh in range(repeat_):
    for ii in range(layers):
        for jj in range(2):
            for kk in range(len(mbins)-1):
                ima_size = int(r180_med_ang[jj,kk]*180/np.pi*60*15/zoom)*4
                pure_count[hh][ii][jj][kk] = np.zeros((ima_size,ima_size))
                weighted_count[hh][ii][jj][kk] = np.zeros((ima_size,ima_size))
                mask_ima[hh][ii][jj][kk] = np.zeros((ima_size,ima_size))
                pure_count_sat[hh][ii][jj][kk] = np.zeros((ima_size,ima_size))
                weighted_count_sat[hh][ii][jj][kk] = np.zeros((ima_size,ima_size))
                mask_ima_sat[hh][ii][jj][kk] = np.zeros((ima_size,ima_size))
                pure_count_sat_bq[hh][ii][jj][kk] = np.zeros((ima_size,ima_size))
                weighted_count_sat_bq[hh][ii][jj][kk] = np.zeros((ima_size,ima_size))
                mask_ima_sat_bq[hh][ii][jj][kk] = np.zeros((ima_size,ima_size))